In [91]:
# numerical fun with Brady, Georgia and Markus USING JULIA
# insert comments and notes in this cell

In [92]:
using Pkg
using PyPlot
using Plots
using Plotly
using DelimitedFiles

In [93]:
# define parameters

# We solve the PDE on $[0,1]^2 \times [0,T]$

N = 100 + 1; # we assume M = N and the index for our spatial points goes 1,...,N, so 2,...,N-1 are the inner spatial points
h = 1/(N-1); # spatial step size

T = 0.1;
N_time = 100000 + 1; # the index for our time points goes 1,...,N_time, so 2,...,N_time-1 are the inner time points
tau = T/(N_time - 1); # temporal step size

beta = 1; # we assume beta to be positive
gamma = 1; # we assume gamma to be positive

println("tau = ", tau)
println("h = ", h)


tau = 1.0e-6
h = 0.01


In [94]:
# define initial and coefficient functions for PDE

function alpha(x,y)
    """alpha must be a continuous and strictly positive function on Omega = [0,1]^2 """
    
    #val = (x-1)^2 + (y-1)^2 + 1
    
    val = 1
    
    return val
    
end

function v_0(x,y)
    """initial function for v at t = 0. we assume that v_0 is C^1 on Omega = [0,1]^2"""
    
    val = (x-0.5)^2 + (y-0.5)^2
    
    #val = 1
    
    return val
end
    

v_0 (generic function with 1 method)

In [95]:
# define auxiliary functions

function cart_coord(m, n, t; h = h, tau = tau)
    """Get cartesian coordinates from indices"""
    
    x = h*(m-1)
    y = h*(n-1)
    tt = tau*(t-1)
    
    return (x,y,tt)
end

function plot_sol(S; h=h)
    x = 0:h:1
    y = x'
    
    z = S
    surf(x,y,z)
end

plot_sol (generic function with 1 method)

In [96]:
# define main function


function solve_pde(;N = N, T = T, N_time = N_time, beta = beta, gamma = gamma, alpha = alpha, v_0 = v_0)
    
    # define spatial and temporal stepsize
    h = 1/(N-1)
    tau = T/(N_time - 1)
    
    # define list, will be filled with N_times matrices, each matrix corresponding to one spatial slice at constant t
    pde_solution = []
    
        # compute solutions by stepping through time
    for t = 1:N_time


        # compute initial values at t = 0 via v_0
        if t == 1
            
            S_current = zeros(N,N)
            
            for m = 1:N, n = 1:N
                (x,y,tt) = cart_coord(m, n, t; h = h, tau = tau)              
                S_current[m,n] = v_0(x,y)
            end
            
            # store current solution
            push!(pde_solution, S_current)
        
            
        else
            
            
            # compute inner spatial points for t = 2
            
            S_current = zeros(N,N)
            S_old = pde_solution[t-1]
            
            for m = 2:N-1, n = 2:N-1
                    
                (x,y,tt) = cart_coord(m, n, t; h = h, tau = tau)
                (x_plus,y_plus,tt) = cart_coord(m + 1, n + 1, t; h = h, tau = tau)
                (x_minus, y_minus, tt) = cart_coord(m - 1, n - 1, t; h = h, tau = tau)
                    
                A = alpha(x,y)*(S_old[m+1,n] + S_old[m-1,n] - 4*S_old[m,n] + S_old[m,n+1] + S_old[m,n-1])
                    
                B = (1/4)*(alpha(x_plus,y) - alpha(x_minus,y))*(S_old[m+1,n] - S_old[m-1,n])
                    
                C = (1/4)*(alpha(x,y_plus) - alpha(x,y_minus))*(S_old[m,n+1]-S_old[m,n-1])
                    
                bracket = A + B + C
                    
                S_current[m,n] = S_old[m,n] + (tau/(gamma*(h^2))) * bracket
                    
            end
            
            
            # compute boundary values using boundary condition
            
            for n = 2:N-1
                (x,y,tt) = cart_coord(0, n, t; h = h, tau = tau)
                S_current[1,n] = -(S_current[3,n] - 4*S_current[2,n])/(3 + (2*h*beta/alpha(0,y)))
                S_current[N,n] = (4*S_current[N-1,n] - S_current[N-2,n])/(3 + (2*h*beta/alpha(1,y)))
            end
            
            for m  = 2:N-1
                (x,y,tt) = cart_coord(m, 0, t; h = h, tau = tau)
                S_current[m,1] = -(S_current[m,3] - 4*S_current[m,2])/(3 + (2*h*beta/alpha(x,0)))
                S_current[m,N] = (4*S_current[m,N-1] - S_current[m,N-2])/(3 + (2*h*beta/alpha(x,1)))
            end
            
            
            # compute values at the four corners averaging their three neighbours
            
            S_current[1,1] = (S_current[1,2] + S_current[2,1] + S_current[2,2])/3
            S_current[1, N] = (S_current[2,N] + S_current[1,N-1] + S_current[2,N-1])/3
            S_current[N,1] = (S_current[N-1,1] + S_current[N,2] + S_current[N-1,2])/3
            S_current[N,N] = (S_current[N-1,N] + S_current[N,N-1] + S_current[N-1,N-1])/3         
            
            #store current solution
            push!(pde_solution, S_current)
            
            
            
            
        end
    end

    return pde_solution

end

solve_pde (generic function with 1 method)

In [ ]:
pde_solution = solve_pde(N = N, T = T, N_time = N_time, beta = beta, gamma = gamma, alpha = alpha, v_0 = v_0);

In [85]:
for t=1:N_time
    if mod(t,1) == 0
        println(t)
        println(pde_solution[t])
        writedlm( string("pde_solution",t,".csv"), pde_solution[t] , ',')
    end
end

1
[0.5 0.41000000000000003 0.33999999999999997 0.29 0.26 0.25 0.26 0.29000000000000004 0.34 0.41000000000000003 0.5; 0.41000000000000003 0.32000000000000006 0.25 0.2 0.17000000000000004 0.16000000000000003 0.17000000000000004 0.20000000000000007 0.25000000000000006 0.32000000000000006 0.41000000000000003; 0.33999999999999997 0.25 0.18 0.12999999999999998 0.09999999999999999 0.09 0.10000000000000002 0.13000000000000003 0.18000000000000002 0.25 0.33999999999999997; 0.29 0.2 0.12999999999999998 0.07999999999999996 0.049999999999999975 0.03999999999999998 0.049999999999999996 0.08000000000000002 0.13 0.2 0.29; 0.26 0.17000000000000004 0.09999999999999999 0.049999999999999975 0.01999999999999999 0.009999999999999995 0.02000000000000001 0.050000000000000024 0.10000000000000002 0.17000000000000004 0.26; 0.25 0.16000000000000003 0.09 0.03999999999999998 0.009999999999999995 0.0 0.010000000000000018 0.04000000000000003 0.09000000000000002 0.16000000000000003 0.25; 0.26 0.17000000000000004 0.100

[0.27416733600260423 0.2727455278320313 0.23419542968750004 0.1924674958496094 0.1654405625 0.15634529174804687 0.16544056250000003 0.19246749584960943 0.23419542968750004 0.2727455278320313 0.27416733600260423; 0.2727455278320313 0.27701095234375006 0.23525812031250004 0.19038885683593754 0.16136272343750002 0.15159638417968752 0.16136272343750005 0.19038885683593756 0.23525812031250004 0.27701095234375006 0.2727455278320313; 0.23419542968750004 0.23525812031250004 0.19160710625 0.145659440625 0.11604109375 0.106080603125 0.11604109375000003 0.14565944062500005 0.19160710625000002 0.23525812031250004 0.23419542968750004; 0.19246749584960937 0.1903888568359375 0.145659440625 0.09942858124999998 0.0697128125 0.059723103124999996 0.06971281250000001 0.09942858125000002 0.14565944062500003 0.1903888568359375 0.19246749584960937; 0.1654405625 0.16136272343750002 0.11604109375 0.06971281249999998 0.03998419999999999 0.029992162499999996 0.039984200000000004 0.06971281250000003 0.11604109375

[0.24729753315836792 0.24520963935365298 0.21748688992678836 0.18185990808996969 0.15656837228176881 0.1477793882182198 0.15656837228176887 0.18185990808996969 0.21748688992678836 0.24520963935365298 0.24729753315836792; 0.24520963935365298 0.2514733207677979 0.221222437139502 0.18273875043545232 0.15547875307697756 0.1460108926593903 0.1554787530769776 0.18273875043545232 0.22122243713950202 0.2514733207677979 0.24520963935365298; 0.21748688992678833 0.22122243713950196 0.1889317007922852 0.14900329585390626 0.12089622100625 0.11114952833925783 0.12089622100625003 0.1490032958539063 0.18893170079228522 0.221222437139502 0.21748688992678836; 0.18185990808996966 0.1827387504354523 0.14900329585390626 0.10844933723779296 0.0800774933330078 0.07025356956323242 0.08007749333300783 0.10844933723779299 0.1490032958539063 0.1827387504354523 0.18185990808996966; 0.15656837228176881 0.15547875307697756 0.12089622100625001 0.0800774933330078 0.05163363785937499 0.041793841440429684 0.05163363785

[0.2198901925309632 0.21734500169678692 0.19865321835135732 0.17045792035437976 0.14838799672367414 0.14034720900631917 0.14838799672367417 0.17045792035437976 0.19865321835135738 0.21734500169678694 0.21989019253096323; 0.21734500169678692 0.2249805741993158 0.20441829136754514 0.17382173084825558 0.14994498392689914 0.14125380648891764 0.14994498392689917 0.1738217308482556 0.2044182913675452 0.22498057419931586 0.21734500169678694; 0.19865321835135732 0.20441829136754514 0.18198286674583708 0.14982157825900713 0.12493834619183929 0.11590415713544924 0.1249383461918393 0.14982157825900716 0.1819828667458371 0.20441829136754516 0.19865321835135735; 0.17045792035437976 0.17382173084825558 0.14982157825900713 0.11674765931228319 0.09139781484284105 0.08222008231672283 0.09139781484284107 0.11674765931228322 0.14982157825900716 0.17382173084825558 0.17045792035437976; 0.14838799672367414 0.14994498392689914 0.12493834619183929 0.09139781484284105 0.06586043414596619 0.0566330351334423 0.

[0.1986938597769565 0.19592276971616918 0.18307183749968442 0.16096422850496747 0.14239435033382533 0.13537176775726362 0.14239435033382533 0.1609642285049675 0.18307183749968445 0.19592276971616923 0.19869385977695653; 0.19592276971616918 0.2042360398985311 0.189991296502383 0.16591289229648515 0.14576250153159265 0.1381509174234878 0.14576250153159265 0.16591289229648518 0.18999129650238303 0.20423603989853115 0.19592276971616923; 0.18307183749968442 0.189991296502383 0.17413530601054186 0.1485660379700446 0.12738808505812949 0.11941401287070766 0.12738808505812949 0.1485660379700446 0.1741353060105419 0.189991296502383 0.1830718374996844; 0.16096422850496747 0.16591289229648512 0.14856603797004458 0.12197551679704163 0.10020956196022343 0.09204431905081868 0.10020956196022344 0.12197551679704165 0.1485660379700446 0.16591289229648515 0.16096422850496747; 0.14239435033382533 0.14576250153159265 0.12738808505812949 0.10020956196022344 0.07815508423985403 0.06990434181235575 0.07815508

...

[0.0926716890876107 0.09006543629350856 0.09507661797177672 0.09816731073975864 0.09976478129513433 0.10024744235923123 0.09976478129513433 0.09816731073975864 0.09507661797177672 0.09006543629350858 0.09267168908761071; 0.09006543629350856 0.097884194675815 0.10332738256403255 0.10668243844190896 0.10841531075450284 0.10893860595694088 0.10841531075450284 0.10668243844190896 0.10332738256403255 0.097884194675815 0.09006543629350856; 0.09507661797177672 0.10332738256403255 0.10906435274644473 0.11259435940040817 0.11441394287358149 0.11496260827822359 0.11441394287358149 0.11259435940040817 0.10906435274644473 0.10332738256403254 0.09507661797177669; 0.09816731073975864 0.10668243844190896 0.11259435940040817 0.1162242037565667 0.11809059042193126 0.1186523262099988 0.11809059042193126 0.1162242037565667 0.11259435940040817 0.10668243844190894 0.09816731073975862; 0.09976478129513432 0.10841531075450284 0.1144139428735815 0.11809059042193128 0.11997719466373385 0.12054415253344125 0.11

[0.09088461992990667 0.0883117075685923 0.09336596458873171 0.09657998549139354 0.09829877933274697 0.09883094275651921 0.09829877933274697 0.09657998549139354 0.09336596458873171 0.0883117075685923 0.09088461992990667; 0.0883117075685923 0.0960304446525354 0.10152431439064619 0.10501645119901842 0.10688314729474586 0.1074609247319198 0.10688314729474586 0.10501645119901842 0.10152431439064619 0.0960304446525354 0.0883117075685923; 0.09336596458873171 0.10152431439064619 0.10732617087864328 0.11100985122361434 0.11297649531419313 0.11358468210681773 0.11297649531419313 0.11100985122361434 0.10732617087864328 0.10152431439064617 0.09336596458873168; 0.09657998549139353 0.10501645119901841 0.11100985122361434 0.11480977098993958 0.11683536615276664 0.11746111255508485 0.11683536615276664 0.11480977098993958 0.11100985122361434 0.10501645119901841 0.09657998549139353; 0.09829877933274697 0.10688314729474586 0.11297649531419313 0.11683536615276664 0.1188898284535472 0.11952393923068957 0.1

[0.08961974592746959 0.08707175654275452 0.09214554380335968 0.09543250944220681 0.09722524629359022 0.09778783617253137 0.09722524629359022 0.09543250944220681 0.09214554380335968 0.08707175654275452 0.08961974592746959; 0.08707175654275452 0.09471572469689973 0.10023327785078445 0.10380664018652695 0.10575496282231021 0.10636624754069383 0.10575496282231021 0.10380664018652695 0.10023327785078445 0.09471572469689973 0.08707175654275452; 0.09214554380335968 0.10023327785078445 0.10606737123238681 0.10984253053104598 0.1118990631497521 0.11254391441067495 0.1118990631497521 0.10984253053104598 0.1060673712323868 0.10023327785078444 0.09214554380335967; 0.0954325094422068 0.10380664018652694 0.10984253053104598 0.11374419804632994 0.11586733689223618 0.1165325844444243 0.11586733689223618 0.11374419804632994 0.10984253053104598 0.10380664018652694 0.0954325094422068; 0.09722524629359022 0.10575496282231021 0.1118990631497521 0.11586733689223618 0.11802482269354948 0.11870043011101823 0.

[0.08841153817912974 0.08588837482637054 0.09097238798855802 0.09431799430028275 0.0961721842803736 0.09676028995441778 0.0961721842803736 0.09431799430028275 0.09097238798855799 0.08588837482637054 0.08841153817912974; 0.08588837482637053 0.09345786488464813 0.09898866009420679 0.10262744934216314 0.10464366413838963 0.10528306416596357 0.10464366413838963 0.10262744934216314 0.09898866009420677 0.09345786488464813 0.08588837482637054; 0.09097238798855799 0.09898866009420677 0.1048429988134415 0.10869221560774776 0.11082366685636302 0.11149932880971737 0.11082366685636302 0.10869221560774776 0.1048429988134415 0.09898866009420677 0.09097238798855799; 0.09431799430028275 0.10262744934216313 0.10869221560774775 0.11267669683305877 0.11488132320188459 0.11557982200333901 0.11488132320188459 0.11267669683305877 0.10869221560774775 0.10262744934216313 0.09431799430028275; 0.0961721842803736 0.10464366413838963 0.11082366685636302 0.11488132320188459 0.11712502195746813 0.1178356051427761 0

[0.08687814477432904 0.0843878982378922 0.0894732458721342 0.09287803071576856 0.09479753591700872 0.09541312399339698 0.09479753591700872 0.09287803071576856 0.0894732458721342 0.0843878982378922 0.08687814477432904; 0.0843878982378922 0.0918586378472027 0.09739327702755582 0.10109829918723369 0.10318675240946838 0.10385645775019976 0.10318675240946838 0.10109829918723369 0.09739327702755582 0.0918586378472027 0.0843878982378922; 0.08947324587213419 0.0973932770275558 0.10325872131939377 0.10718349845847534 0.10939489470344561 0.11010383422192871 0.10939489470344561 0.10718349845847534 0.10325872131939377 0.0973932770275558 0.08947324587213419; 0.09287803071576856 0.10109829918723369 0.10718349845847534 0.11125317464311103 0.11354503591391141 0.11427953025759525 0.11354503591391141 0.11125317464311103 0.10718349845847534 0.10109829918723369 0.09287803071576856; 0.09479753591700872 0.10318675240946838 0.10939489470344561 0.11354503591391142 0.11588124714174827 0.1166297572554089 0.1158

In [ ]:
# define main function


def solve_pde(N = N, T = T, N_time = N_time, beta = beta, gamma = gamma, alpha = alpha, v_0 = v_0):
    
    # define spatial and temporal stepsize
    h = 1/(N+1)
    tau = T/(N_time + 1)
    
    # preallocate list, will be filled with matrices, each matrix corresponding to one spatial slice at constant t
    pde_solution = [None]*(N_time + 2)
    
    # compute solutions by stepping through time
    for t in range(N_time + 2):
        
        
        # compute initial values at t = 0 via v_0
        if t == 0:
            S_current = np.zeros((N+2,N+2))
            for m in range(N+2):
                for n in range(N+2):
                    (x,y,tt) = cart_coord(m = m, n = n, t = t, h = h, tau = tau)
                    S_current[m,n] = v_0(x,y)
            
            # save current solution
            pde_solution[t] = S_current
            
        
        # initialize our scheme with forward difference quotient in time
        elif t == 1:
            
            # compute inner spatial points for t = 1
            
            S_current = np.zeros((N+2,N+2))
            S_old = pde_solution[t-1]
            
            for m in range(1,N+1):
                for n in range(1,N+1):
                    
                    (x,y,tt) = cart_coord(m = m, n = n, t = t, h = h, tau = tau)
                    (x_plus,y_plus,tt) = cart_coord(m = m + 1, n = n + 1, t = t, h = h, tau = tau)
                    (x_minus, y_minus, tt) = cart_coord(m = m - 1, n = n - 1, t = t, h = h, tau = tau)
                    
                    A = alpha(x,y)*(S_old[m+1,n] + S_old[m-1,n] - 4*S_old[m,n] + S_old[m,n+1] + S_old[m,n-1])
                    
                    B = (1/4)*(alpha(x_plus,y) - alpha(x_minus,y))*(S_old[m+1,n] - S_old[m-1,n])
                    
                    C = (1/4)*(alpha(x,y_plus) - alpha(x,y_minus))*(S_old[m,n+1]-S_old[m,n-1])
                    
                    bracket = A + B + C
                    
                    S_current[m,n] = S_old[m,n] + (tau/(gamma*(h**2))) * bracket
                    
            
            # compute boundary values using boundary condition
            
            for n in range(1,N+1):
                (x,y,tt) = cart_coord(m = 0, n = n, t = t, h = h, tau = tau)
                S_current[0,n] = -(S_current[2,n] - 4*S_current[1,n])/(3 + (2*h*beta/alpha(0,y)))
                S_current[N+1,n] = (4*S_current[N,n] - S_current[N-1,n])/(3 + (2*h*beta/alpha(1,y)))
            
            for m in range(1,N+1):
                (x,y,tt) = cart_coord(m = m, n = 0, t = t, h = h, tau = tau)
                S_current[m,0] = -(S_current[m,2] - 4*S_current[m,1])/(3 + (2*h*beta/alpha(x,0)))
                S_current[m,N+1] = (4*S_current[m,N] - S_current[m,N-1])/(3 + (2*h*beta/alpha(x,1)))
            
            
            # compute values at the four corners averaging their three neighbours
            
            S_current[0,0] = (S_current[0,1] + S_current[1,0] + S_current[1,1])/3
            S_current[0, N+1] = (S_current[1,N+1] + S_current[0,N] + S_current[1,N])/3
            S_current[N+1,0] = (S_current[N,0] + S_current[N+1,1] + S_current[N,1])/3
            S_current[N+1,N+1] = (S_current[N,N+1] + S_current[N+1,N] + S_current[N,N])/3
            
            # save current solution
            pde_solution[t] = S_current
        
        
        else:
                        
            # compute inner spatial points for t (with t>1)
            
            S_current = np.zeros((N+2,N+2))
            S_old = pde_solution[t-1]
            S_very_old = pde_solution[t-2]
            
            for m in range(1,N+1):
                for n in range(1,N+1):
                    
                    (x,y,tt) = cart_coord(m = m, n = n, t = t, h = h, tau = tau)
                    (x_plus,y_plus,tt) = cart_coord(m = m + 1, n = n + 1, t = t, h = h, tau = tau)
                    (x_minus, y_minus, tt) = cart_coord(m = m - 1, n = n - 1, t = t, h = h, tau = tau)
                    
                    A = alpha(x,y)*(S_old[m+1,n] + S_old[m-1,n] - 4*S_old[m,n] + S_old[m,n+1] + S_old[m,n-1])
                    
                    B = (1/4)*(alpha(x_plus,y) - alpha(x_minus,y))*(S_old[m+1,n] - S_old[m-1,n])
                    
                    C = (1/4)*(alpha(x,y_plus) - alpha(x,y_minus))*(S_old[m,n+1]-S_old[m,n-1])
                    
                    bracket = A + B + C
                    
                    S_current[m,n] = S_very_old[m,n] + ((2*tau)/(gamma*(h**2))) * bracket
                    
            
            # compute boundary values using boundary condition
            
            for n in range(1,N+1):
                (x,y,tt) = cart_coord(m = 0, n = n, t = t, h = h, tau = tau)
                S_current[0,n] = -(S_current[2,n] - 4*S_current[1,n])/(3 + (2*h*beta/alpha(0,y)))
                S_current[N+1,n] = (4*S_current[N,n] - S_current[N-1,n])/(3 + (2*h*beta/alpha(1,y)))
            
            for m in range(1,N+1):
                (x,y,tt) = cart_coord(m = m, n = 0, t = t, h = h, tau = tau)
                S_current[m,0] = -(S_current[m,2] - 4*S_current[m,1])/(3 + (2*h*beta/alpha(x,0)))
                S_current[m,N+1] = (4*S_current[m,N] - S_current[m,N-1])/(3 + (2*h*beta/alpha(x,1)))
            
            
            # compute values at the four corners averaging their three neighbours
            
            S_current[0,0] = (S_current[0,1] + S_current[1,0] + S_current[1,1])/3
            S_current[0, N+1] = (S_current[1,N+1] + S_current[0,N] + S_current[1,N])/3
            S_current[N+1,0] = (S_current[N,0] + S_current[N+1,1] + S_current[N,1])/3
            S_current[N+1,N+1] = (S_current[N,N+1] + S_current[N+1,N] + S_current[N,N])/3
            
            # save current solution
            pde_solution[t] = S_current

    return pde_solution